### Linear regression
We'll create model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall and humidity (input variables or features) in a region.

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some connstant, known as a bias:

yield_apple = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b1

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:


### Training data
The training data can be represented using 2 matrices: inputs and targets, each with one row per observation, and one column per variable.

In [1]:
# Input (temp, rainfall, humidity)
import numpy as np
import torch
inputs = np.array([[73, 67, 43], 
                 [91, 88, 64],
                 [87, 134, 58],
                 [102, 43, 37],
                 [69, 96, 70]], dtype = 'float32')

In [2]:
# Targets (apples, oranges)
targets = np.array([[56, 70],
                   [81, 101],
                   [119, 133],
                   [22, 37],
                   [109, 119]], dtype = 'float32')

We've seperated the input and target variables, because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors as follows:

In [3]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [109., 119.]])


### Linear regression from skratch
The weights and biases (w11, w12,..., w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [4]:
# Weights and biases
w = torch.randn(2, 3, requires_grad = True)
b = torch.randn(2, requires_grad = True)
print(w)
print(b)

tensor([[ 1.2037,  0.0959,  1.1775],
        [-0.6804,  0.4350,  2.0824]], requires_grad=True)
tensor([-0.6910, -1.0433], requires_grad=True)


torch.randn creates a tensor wiith a given shape, with elements picked randomly from a normal distribution ith mean 0 and standard deviation 1.

Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation).
$$X * W^T + b$$

In [5]:
def model(x):
    return x @ w.t() + b

@ represents matrix multiplication in PyTorch, and the .t method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [6]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[144.2309,  67.9802],
        [192.6370, 108.6000],
        [185.1670, 118.8366],
        [169.7719,  25.3151],
        [173.9880, 139.5427]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [7]:
# Compare with the targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [109., 119.]])


You can see that there is a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to just work.

### Loss function
Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

Calculate the diffrence between the two matrices (preds and targets).

Square all elements of the diffrence matrix to remove negative values.

Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the mean square error (MSE).

In [8]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel() # numel = number of elements

torch.sum returns the sum of all the elements in a tensor, and the .numel method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [9]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(5150.6562, grad_fn=<DivBackward0>)


Here's how we can interpret the result: On average, each element in the prediction differs from the actual target by about 138 (square root of the loss 19044). And that's pretty bad, considering the numbers we are trying to predictare themselves in the range 50-200. Also, the result is called the loss, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

### Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have requires_grad set to True.

In [10]:
# Compute gradients
loss.backward()

C:\Users\Agata\Anaconda3\lib\site-packages\torch\autograd\__init__.py:145: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(


The gradients are stored in the .grad propety of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix, with he same dimensions.

In [11]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 1.2037,  0.0959,  1.1775],
        [-0.6804,  0.4350,  2.0824]], requires_grad=True)
tensor([[8382.6533, 7438.9912, 4958.6221],
        [ -92.4932,   21.0468,  116.7449]])


In [12]:
print(b)
print(b.grad)

tensor([-0.6910, -1.0433], requires_grad=True)
tensor([9.5759e+01, 5.4940e-02])


The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t. any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

If gradient element is positive:
Increesing the element's value slightly will increas the loss.
Decreasing the element's value slightly will decrease the loss.

If gradient element is positive:
Increasing the element's value slightly will decrease the loss.
Decrasing the element's value slightly will increase the loss.

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimizaion algorithm that we'll use to improve our model.


Before we proceed, we reset the gradients to zero by calling .zero_() method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call .backward on the loss, the new gradient values will get added to the exising gradient values, which may lead to unexpected results.

In [13]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


### Adjust weights and biases using gardient descent
We'll reduce the loss and improve ouur model using the gradient decent optimization algorithm, which has the following steps:
1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weihts by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

Let's implement the above step by step.

In [14]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[144.2309,  67.9802],
        [192.6370, 108.6000],
        [185.1670, 118.8366],
        [169.7719,  25.3151],
        [173.9880, 139.5427]], grad_fn=<AddBackward0>)


Note that the predictions are same as before, since we haven't made any changes to our model. The same holds true for the loss and gradients.

In [15]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(5150.6562, grad_fn=<DivBackward0>)


In [16]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[8382.6533, 7438.9912, 4958.6221],
        [ -92.4932,   21.0468,  116.7449]])
tensor([9.5759e+01, 5.4940e-02])


Finally, we update the weights and biases using the gradients computed above.

In [17]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:
- We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases.
- We multiply the gradients with a really small number (10^-5 in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. this number is called the learning rate of the algorithm.
- After we have updated the weights, we reset the gradients bach to zero, to avoid affecting any future computations.

Let's take a look at the new weughts and biases.

In [18]:
print(w)
print(b)

tensor([[ 1.1198,  0.0215,  1.1279],
        [-0.6794,  0.4348,  2.0813]], requires_grad=True)
tensor([-0.6920, -1.0433], requires_grad=True)


With the new weights and biases, the model should have lower loss.

In [19]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)  # mse (mean square error) -> standardna devijacija,
print(loss)                 # prva variabla je kao aritmeticka sredina a druga je nase mjerenje

tensor(3781.9011, grad_fn=<DivBackward0>)


We have already achived a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

### Train for multiple epochs
To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.  Let's train the model for 100 epochs.

In [20]:
# Train for 100  epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5      # -= je kao -- u c++
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [21]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(318.4440, grad_fn=<DivBackward0>)


In [22]:
# Predictions
preds

tensor([[ 64.5053,  68.5894],
        [ 91.1558, 108.2035],
        [ 92.5610, 118.5865],
        [ 60.0458,  28.9114],
        [ 94.1761, 136.7666]], grad_fn=<AddBackward0>)

In [23]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [109., 119.]])

The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs.

In [24]:
# Train for 100  epochs
for i in range(500):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-4      # -= je kao -- u c++
        b -= b.grad * 1e-4
        w.grad.zero_()
        b.grad.zero_()
        
preds = model(inputs)
loss = mse(preds, targets)

print(loss)
print(preds)
print(targets)

tensor(2.8179, grad_fn=<DivBackward0>)
tensor([[ 57.6371,  70.1333],
        [ 84.6273, 100.7812],
        [118.4503, 132.9852],
        [ 19.9401,  37.0369],
        [106.2223, 119.1115]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [109., 119.]])
